In [ ]:
# Sorting just with Adjacent Swaps : O(n^2)

A=[10,9,8,7,6,5,5,4,3,2,1,1,0,-1]
n=len(A)

for i in range(n-1):
  for j in range(i+1,n):
     if (A[i]>A[j]):
        # Swap elements if they are in the wrong order:
        A[i], A[j] = A[j], A[i]

# print the sorted array:
print(A)

[-1, 0, 1, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10]


# Count Adjacent swaps : MERGE SORT Count

In [ ]:

def count_inversions(A): # A = array of length n
    n = len(A)

    #Base Case:
    if n <= 1:
        return A, 0
    #Divide the array into two equal parts:
    mid = n //2 # middle index

    ##Apply count_inversion on left and right part recursively:
    # sorted_left == sorted left half array
    # count_inv_left == number of inversions required for sorting
    sorted_left, count_inv_left = count_inversions(A[:mid])
    sorted_right, count_inv_right = count_inversions(A[mid:])

    # Merged_arr= merged and sorted array
    # count_cross_inv=number of inversions where one element is from the left half and the other from the right half.
    merged_arr, count_cross_inv = mergeSort_count(sorted_left, sorted_right)

    # Total number of swaps :
    total_inv = count_inv_left + count_inv_right + count_cross_inv

    return merged_arr, total_inv

#Function to MergeSort two arrays and count the cross_inversion btw them
def mergeSort_count(left, right):
    #Input : left-half and right-half of the array
    #Output : merged array and cross_inversion_count

    i = 0 #left
    j = 0 #right
    cross_inv_count = 0 # number of cross inversion accumulator
    merged = [] # final sorted array

    while i < len(left) and j < len(right):
        if left[i] <= right[j]: # If the current element in left is less than or equal to the current element in right :
            merged.append(left[i]) # Append left[i] to the merged list (it belongs next in sorted order).
            i += 1 # Advance the pointer in the left list.
        else: # Otherwise left[i] > right[j] — we found a cross inversion.
            merged.append(right[j])  # Append right[j] (the smaller item) to the merged list.
            j += 1  # Advance the pointer in the right list.

            cross_inv_count += len(left) - i  # all remaining left elements are greater
            ## Counting step explanation:
            #since left and right are sorted, if left[i] > right[j],
            #then every element left[i], left[i+1], ... is also greater than right[j].
            #So we add len(left) - i to the inversion count — these are all the cross-inversion pairs involving right[j].

    # Once the while loop ends, one of the lists may still have items.
    # So we,append any remaining elements from left to merged.
    # No new cross inversions are introduced here because the remaining items are already larger (or smaller) in order.
    #print(i,j)
    merged += left[i:]
    merged += right[j:]
    return merged, cross_inv_count

## Example from the link:
arr = [3,5,1,10,9,2,6,8]
Inversions = count_inversions(arr)
#print(Inversions)
print(f"Number of swaps = {Inversions[1]}")
print(f"Sorted array ={Inversions[0]}")


Number of swaps = 11
Sorted array =[1, 2, 3, 5, 6, 8, 9, 10]


## Algorithm : Being Unique
### Task:
You are given an array A of n numbers from the set {1, 2, . . . , n}. The array has the “unique-in-range”
property if for every range [i, j] there exists an element A[k] (where i ≤k ≤j) such that the number A[k]
occurs just once in that range.

**Explanation**:
For example, the sequence: 1 2 1 3 1 2 1 4 1 2 1 3 1 has the unique-in-range property. As does 1 2 3 4 5 6. But the sequence 1 2 3 1 2 3 does not (it fails on the whole range: every number appears twice).
Give an algorithm to determine if a given array has the property. It should have runtime O(n log n) or
better. Prove the runtime and the correctness of your algorithm.

In [ ]:
a = [1 ,2 ,3 ,1]
seen = set()

# Check for duplicates
unique = True
for x in a:
    if x in seen:
        unique = False
        break
    seen.add(x)
    print(seen)

print(unique)

{1}
{1, 2}
{1, 2, 3}
False


# Amortized Analysis Bucket method:

**Reasoning**
* For each element A[i] we compute the interval [l_i, r_i] in which A[i] would be unique (no other occurrences inside). If the union of all those intervals covers every index 1..n, then every subarray contains at least one unique element; otherwise some subarray lacks a unique element.

In [ ]:
def has_unique_in_range(A):
    n = len(A)

    # Step 1: compute previous occurrence of each value
    prev = [0] * n
    last_occ = {}
    for i in range(n):
        if A[i] in last_occ:
            prev[i] = last_occ[A[i]]
        else:
            prev[i] = 0
        last_occ[A[i]] = i + 1  # store 1-based index for convenience

    # Step 2: compute next occurrence of each value
    next_occ = [0] * n
    last_occ.clear()
    for i in range(n - 1, -1, -1):
        if A[i] in last_occ:
            next_occ[i] = last_occ[A[i]]
        else:
            next_occ[i] = n + 1
        last_occ[A[i]] = i + 1

    # Step 3: build bucket lists based on left endpoint l = prev[i] + 1
    B = [[] for _ in range(n + 2)]  # 1-based indexing
    for i in range(n):
        l = prev[i] + 1
        r = next_occ[i] - 1
        B[l].append(r)

    # Step 4: sweep through positions 1..n
    current_max_right = 0
    for pos in range(1, n + 1):
        for r in B[pos]:
            current_max_right = max(current_max_right, r)
        if current_max_right < pos:
            # uncovered position -> fails unique-in-range property
            return False
    return True


# -------------------------
# Example runs
# -------------------------
A1 = [1, 2, 1, 3, 1, 2, 1]  # Has unique-in-range property
A2 = [1, 2, 3, 1, 2, 3]     # Does NOT have the property

print("A1:", has_unique_in_range(A1))  # True
print("A2:", has_unique_in_range(A2))  # False


A1: True
A2: True


# Unique in Range Set method:

In [ ]:

def has_unique_in_range_property_set(arr: list[int]) -> bool:
    """
    Determines if the array has the unique-in-range property using an O(n^2) approach.

    This method checks every possible sub-array. For each sub-array, it uses
    two sets to determine if at least one unique element exists.
    """
    n = len(arr)
    if n == 0:
        return True

    # Iterate through all possible start points of a sub-array
    for i in range(n):
        uniques = set()
        duplicates = set()

        # Expand the sub-array to the right from the start point i
        for j in range(i, n):
            # Current sub-array is arr[i:j+1]
            elem = arr[j]

            if elem not in duplicates:
                if elem in uniques:
                    # It was unique, but we found a second occurrence.
                    # Move it from the uniques set to the duplicates set.
                    uniques.remove(elem)
                    duplicates.add(elem)
                else:
                    # First time we've seen this element in this sub-array.
                    uniques.add(elem)

            # After adding the element, check if the 'uniques' set is empty.
            # If it is, the current sub-array arr[i:j+1] has no unique elements.
            if not uniques:
                # We found a "bad" range, so the property fails.
                # print(f"Failed on range: {arr[i:j+1]}") # Uncomment for debugging
                return False

    # If we check all possible sub-arrays and never find a "bad" one, the property holds.
    return True

# --- Example Usage ---
if __name__ == "__main__":

    # Example 1 (from problem): Should be True
    arr1 = [1, 2, 1, 3, 1, 2, 1, 4, 1, 2, 1, 3, 1]
    print(f"Array: {arr1}")
    print(f"Has property: {has_unique_in_range_property_set(arr1)}\n")

    # Example 2 (from problem): Should be True
    arr2 = [1, 2, 3, 4, 5]
    print(f"Array: {arr2}")
    print(f"Has property: {has_unique_in_range_property_set(arr2)}\n")

    # Example 3 (from problem): Should be False
    arr3 = [1, 2, 3, 1, 2, 3]
    print(f"Array: {arr3}")
    print(f"Has property: {has_unique_in_range_property_set(arr3)}\n")

    # Example 4: Should be False (the range [1,2,1,2] is bad)
    arr4 = [1, 2, 1, 2]
    print(f"Array: {arr4}")
    print(f"Has property: {has_unique_in_range_property_set(arr4)}\n")

Array: [1, 2, 1, 3, 1, 2, 1, 4, 1, 2, 1, 3, 1]
Has property: True

Array: [1, 2, 3, 4, 5]
Has property: True

Array: [1, 2, 3, 1, 2, 3]
Has property: False

Array: [1, 2, 1, 2]
Has property: False



#Notes on Decision Tree Sorting:
Explain how a decision tree can be used to visualize a sorting algorithm that uses adjacent swaps.

### Explain the concept

### Subtask:
Describe what a decision tree for a sorting algorithm represents.

**Reasoning**:
Describe what a decision tree for a sorting algorithm represents, covering the points about nodes, branches, leaves, and paths.


* A decision tree for a comparison-based sorting algorithm visually represents all possible sequences of comparisons that the algorithm might perform for a given input size.

* Each internal node in the tree corresponds to a comparison between two elements.

* The branches from an internal node represent the two possible outcomes of the comparison (e.g., element A is less than element B, or element A is greater than or equal to element B).
* The leaves of the tree represent the sorted permutations of the input array, and the path from the root to a leaf represents a specific execution of the sorting algorithm for a particular input order.
* For an algorithm using adjacent swaps, the comparisons will primarily involve adjacent elements, and swaps will occur based on the comparison outcomes, leading to a specific path in the tree.

### Illustrate with a small example

### Subtask:
Use a small array to show how the tree branches based on comparisons and how swaps change the array's state.

**Reasoning**:
Manually simulate the adjacent swap sorting algorithm on the small array [3, 1, 2], describing each comparison, its outcome (branch), and the resulting array state (node).



In [ ]:
# Initial array
arr = [3, 1, 2]
print(f"Initial array: {arr}")

# First pass
print("\nPass 1:")
# Compare arr[0] and arr[1]
print("Node: Compare arr[0] and arr[1] (3 and 1)")
if arr[0] > arr[1]:
    print("Branch: 3 > 1 is True. Swap arr[0] and arr[1].")
    arr[0], arr[1] = arr[1], arr[0]
    print(f"Array after swap: {arr}")
else:
    print("Branch: 3 > 1 is False. No swap.")
print(f"Array after comparison: {arr}")

# Compare arr[1] and arr[2]
print("\nNode: Compare arr[1] and arr[2] (3 and 2)")
if arr[1] > arr[2]:
    print("Branch: 3 > 2 is True. Swap arr[1] and arr[2].")
    arr[1], arr[2] = arr[2], arr[1]
    print(f"Array after swap: {arr}")
else:
    print("Branch: 3 > 2 is False. No swap.")
print(f"Array after comparison: {arr}")


# Second pass (for bubble sort, we need n-1 passes in worst case)
print("\nPass 2:")
# Compare arr[0] and arr[1]
print("Node: Compare arr[0] and arr[1] (1 and 2)")
if arr[0] > arr[1]:
    print("Branch: 1 > 2 is True. Swap arr[0] and arr[1].")
    arr[0], arr[1] = arr[1], arr[0]
    print(f"Array after swap: {arr}")
else:
    print("Branch: 1 > 2 is False. No swap.")
print(f"Array after comparison: {arr}")

# Check if sorted (optional, but helps understand termination)
is_sorted = True
for i in range(len(arr) - 1):
    if arr[i] > arr[i+1]:
        is_sorted = False
        break
if is_sorted:
    print("\nArray is sorted.")
else:
    print("\nArray is not yet sorted.")

print(f"Final array state for this path: {arr}")

# Explain how different initial arrays lead to different paths
print("\nExplanation of different paths:")
print("The comparisons made by the algorithm are fixed for a given pass (e.g., compare index 0 and 1, then 1 and 2, etc.). However, the OUTCOME of these comparisons depends on the initial order of the elements.")
print("For example, if the initial array was [1, 2, 3], the comparison 3 > 1 would be False, leading down a different branch (the 'no swap' branch) at that node.")
print("Each unique initial permutation of the array will follow a specific sequence of comparison outcomes, tracing a unique path from the root of the decision tree to one of the leaf nodes (which represents a sorted permutation).")

Initial array: [3, 1, 2]

Pass 1:
Node: Compare arr[0] and arr[1] (3 and 1)
Branch: 3 > 1 is True. Swap arr[0] and arr[1].
Array after swap: [1, 3, 2]
Array after comparison: [1, 3, 2]

Node: Compare arr[1] and arr[2] (3 and 2)
Branch: 3 > 2 is True. Swap arr[1] and arr[2].
Array after swap: [1, 2, 3]
Array after comparison: [1, 2, 3]

Pass 2:
Node: Compare arr[0] and arr[1] (1 and 2)
Branch: 1 > 2 is False. No swap.
Array after comparison: [1, 2, 3]

Array is sorted.
Final array state for this path: [1, 2, 3]

Explanation of different paths:
The comparisons made by the algorithm are fixed for a given pass (e.g., compare index 0 and 1, then 1 and 2, etc.). However, the OUTCOME of these comparisons depends on the initial order of the elements.
For example, if the initial array was [1, 2, 3], the comparison 3 > 1 would be False, leading down a different branch (the 'no swap' branch) at that node.
Each unique initial permutation of the array will follow a specific sequence of comparison o

### Discuss limitations

### Subtask:
Explain why a full decision tree for larger arrays becomes impractical to draw.

**Reasoning**:
Explain the reasons why drawing a full decision tree for larger arrays is impractical, following the instructions.


* A full decision tree for a sorting algorithm on an array of size n must account for all n! possible initial permutations of the elements. Each of these permutations will ideally correspond to a unique path from the root to a leaf node. Therefore, the number of leaf nodes in the decision tree grows factorially with n (n!). This means even for a small increase in array size, the number of required leaf nodes (and thus branches) explodes.

* Each path from the root to a leaf in the decision tree represents the specific sequence of comparisons and potential swaps performed by the algorithm for a particular initial arrangement of the input array. The internal nodes of the tree represent the comparisons made. The number of comparisons required can also be substantial, especially for algorithms like bubble sort in the worst case (O(n^2)). This large number of paths and internal nodes makes the tree extremely complex, wide, and deep.

* Attempting to draw or visualize such a massive tree is computationally infeasible. For example, an array of size 10 has 10! = 3,628,800 permutations. Representing millions of leaf nodes and the corresponding paths and comparison nodes is beyond the capacity of standard visualization tools and is practically impossible for a human to comprehend or analyze visually, even for relatively small arrays (e.g., n > 10).

* In conclusion, while the decision tree is a powerful theoretical concept for understanding the fundamental limitations of comparison-based sorting algorithms and proving lower bounds on the number of comparisons required (such as the O(n log n) comparison lower bound), it is not a practical tool for visualizing the execution of sorting algorithms on real-world datasets of significant size due to its immense complexity.

### Suggest alternative visualizations

### Subtask:
Suggest other ways to visualize sorting algorithms (e.g., step-by-step execution).

**Reasoning**:
Generate text to suggest alternative visualization methods for sorting algorithms, incorporating all the specified instructions.

* Instead of a full decision tree, sorting algorithms can be visualized through step-by-step execution. This shows the state of the array after each comparison and potential swap, providing a clear trace of how the algorithm transforms the input into a sorted output.

* Another effective method is using animated visualizations. These show elements changing position and color to indicate their current state – for example, highlighting elements being compared, showing them being swapped, or marking elements that are already in their sorted position.

* A common and intuitive animation technique uses bar charts. The height of each bar represents the value of an element, and the animation shows the bars rearranging themselves as the sorting algorithm progresses. This provides a dynamic view of the sorting process.

* These alternative visualizations, particularly step-by-step execution and animations, are far more practical for understanding the algorithm's process on larger datasets compared to a full decision tree. A decision tree quickly becomes impractically large and complex as the dataset size increases, making it impossible to follow visually. Step-by-step and animated visualizations, however, focus on the dynamic changes within the array itself, making the sorting process comprehensible even for larger inputs.


### Summary:

### Data Analysis Key Findings

*   A decision tree for a comparison-based sorting algorithm represents all possible sequences of comparisons for a given input size, with internal nodes representing comparisons and leaves representing sorted permutations.
*   For sorting algorithms using adjacent swaps, the comparisons primarily involve adjacent elements, and the resulting swaps determine the path taken through the decision tree for a specific input.
*   Simulating the sorting process on a small array (e.g., `[3, 1, 2]`) demonstrates how comparisons lead to branches in the conceptual tree and how swaps change the array's state, tracing a specific path to the sorted state `[1, 2, 3]`.
*   Different initial array permutations will lead to different comparison outcomes, resulting in different paths through the same decision tree structure.
*   Drawing a full decision tree becomes impractical for even moderately sized arrays (e.g., n > 10) because the number of possible permutations (leaf nodes) grows factorially (n!), leading to an extremely complex and large tree that is computationally infeasible to visualize and impossible for a human to comprehend.

### Insights or Next Steps

*   While decision trees are valuable theoretical tools for understanding sorting algorithm complexity and proving lower bounds, they are not practical for visualizing the algorithm's execution on larger datasets.
*   Alternative visualization methods like step-by-step execution, animated visualizations (including bar charts showing element rearrangement), are more practical and effective for understanding how sorting algorithms work on larger inputs by focusing on the dynamic changes within the array.


In [ ]:
# Initial array
arr = [4, 3, 2, 2,1]
print(f"Initial array: {arr}")

# First pass
print("\nPass 1:")
# Compare arr[0] and arr[1]
print("Node: Compare arr[0] and arr[1] (3 and 1)")
if arr[0] > arr[1]:
    print("Branch: 3 > 1 is True. Swap arr[0] and arr[1].")
    arr[0], arr[1] = arr[1], arr[0]
    print(f"Array after swap: {arr}")
else:
    print("Branch: 3 > 1 is False. No swap.")
print(f"Array after comparison: {arr}")

# Compare arr[1] and arr[2]
print("\nNode: Compare arr[1] and arr[2] (3 and 2)")
if arr[1] > arr[2]:
    print("Branch: 3 > 2 is True. Swap arr[1] and arr[2].")
    arr[1], arr[2] = arr[2], arr[1]
    print(f"Array after swap: {arr}")
else:
    print("Branch: 3 > 2 is False. No swap.")
print(f"Array after comparison: {arr}")


# Second pass (for bubble sort, we need n-1 passes in worst case)
print("\nPass 2:")
# Compare arr[0] and arr[1]
print("Node: Compare arr[0] and arr[1] (1 and 2)")
if arr[0] > arr[1]:
    print("Branch: 1 > 2 is True. Swap arr[0] and arr[1].")
    arr[0], arr[1] = arr[1], arr[0]
    print(f"Array after swap: {arr}")
else:
    print("Branch: 1 > 2 is False. No swap.")
print(f"Array after comparison: {arr}")

# Check if sorted (optional, but helps understand termination)
is_sorted = True
for i in range(len(arr) - 1):
    if arr[i] > arr[i+1]:
        is_sorted = False
        break
if is_sorted:
    print("\nArray is sorted.")
else:
    print("\nArray is not yet sorted.")

print(f"Final array state for this path: {arr}")

Initial array: [4, 3, 2, 2, 1]

Pass 1:
Node: Compare arr[0] and arr[1] (3 and 1)
Branch: 3 > 1 is True. Swap arr[0] and arr[1].
Array after swap: [3, 4, 2, 2, 1]
Array after comparison: [3, 4, 2, 2, 1]

Node: Compare arr[1] and arr[2] (3 and 2)
Branch: 3 > 2 is True. Swap arr[1] and arr[2].
Array after swap: [3, 2, 4, 2, 1]
Array after comparison: [3, 2, 4, 2, 1]

Pass 2:
Node: Compare arr[0] and arr[1] (1 and 2)
Branch: 1 > 2 is True. Swap arr[0] and arr[1].
Array after swap: [2, 3, 4, 2, 1]
Array after comparison: [2, 3, 4, 2, 1]

Array is not yet sorted.
Final array state for this path: [2, 3, 4, 2, 1]


### Discuss complexity

Let's analyze the time complexity of the `MergeSort_count` algorithm.

In [ ]:
print("The `merge_sort_and_count_inversions` function follows the divide-and-conquer paradigm, similar to merge sort.")
print("1. **Divide**: The array is divided into two halves recursively. This takes O(1) time at each level.")
print("2. **Conquer**: The two halves are sorted and inversions are counted recursively. This leads to the recursive calls in the function.")
print("3. **Combine**: The `merge_and_count_split_inversions` function merges the two sorted halves and counts the split inversions. This merging process takes O(n) time for an array of size n, because each element is visited exactly once during the merge.")
print("\nThe recurrence relation for the time complexity of this algorithm is T(n) = 2T(n/2) + O(n).")
print("According to the Master Theorem, this recurrence relation solves to O(n log n).")
print("\nTherefore, the algorithm computes the minimum number of adjacent swaps (inversions) in O(n log n) time.")

The `merge_sort_and_count_inversions` function follows the divide-and-conquer paradigm, similar to merge sort.
1. **Divide**: The array is divided into two halves recursively. This takes O(1) time at each level.
2. **Conquer**: The two halves are sorted and inversions are counted recursively. This leads to the recursive calls in the function.
3. **Combine**: The `merge_and_count_split_inversions` function merges the two sorted halves and counts the split inversions. This merging process takes O(n) time for an array of size n, because each element is visited exactly once during the merge.

The recurrence relation for the time complexity of this algorithm is T(n) = 2T(n/2) + O(n).
According to the Master Theorem, this recurrence relation solves to O(n log n).

Therefore, the algorithm computes the minimum number of adjacent swaps (inversions) in O(n log n) time.
